# Analisis de Exploracion de Datos

#### Paso 1: Importando libreria y configuraciones

In [ ]:
import  pandas as pd 
import plotly.express as px
from utils.FileHandler import FileHandler


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#### Paso 2: Cargando el archivo xls 

In [ ]:
file = FileHandler()
cheatsheet = "Índices aperturas"
xls_name = "sh_ipc_aperturas.xls"

xls = file.get_xls("https://www.indec.gob.ar/ftp/cuadros/economia/sh_ipc_aperturas.xls")
cs = file.get_cheatsheet(xls_name,cheatsheet)

#### Paso 3: Cargando las partes o regiones del archivo xls

Se procede a cargar los lotes del archivo xls, especificamente de la pestaña ‘Índices aperturas’

In [ ]:

regions_name = ["Región_GBA", 
                "Región_Pampeana", 
                "Región_Noroeste", 
                "Región_Noreste", 
                "Región_Cuyo",
                "Región_Patagonia"]

description_region_principal= ["Nivel general",
                    "Alimentos y bebidas no alcohólicas",
                    "Bebidas alcohólicas y tabaco",
                    "Prendas de vestir y calzado",
                    "Vivienda, agua, electricidad, gas y otros combustibles",
                    "Equipamiento y mantenimiento del hogar",
                    "Salud",
                    "Transporte",
                    "Comunicación",
                    "Recreación y cultura",
                    "Educación",
                    "Restaurantes y hoteles",
                    "Bienes y servicios varios"]

df_GBA = pd.read_excel("data/{}.xlsx".format(regions_name[0]),parse_dates=[0])
df_Pampeana = pd.read_excel("data/{}.xlsx".format(regions_name[1]), parse_dates=[0])
df_Noroeste = pd.read_excel("data/{}.xlsx".format(regions_name[2]), parse_dates=[0])
df_Noreste = pd.read_excel("data/{}.xlsx".format(regions_name[3]), parse_dates=[0])
df_Cuyo = pd.read_excel("data/{}.xlsx".format(regions_name[4]), parse_dates=[0])
df_Patagonia =pd.read_excel("data/{}.xlsx".format(regions_name[5]), parse_dates=[0])

#### Paso 4: Identificando las propiedades de los dataframes

Se usara como el ejemplo el archivo con la informacion de la Region GBA.

In [ ]:
df_GBA.info()
#df_Pampeana.info(verbose=False)
#df_Noroeste.info(verbose=False)
#df_Noreste.info(verbose=False)
#df_Cuyo.info(verbose=False)
#df_Patagonia.info(verbose=False)

Se Evidencia que entre los tipos de columnas, hay varias con 
el tipo object... De acuerdo a una revision superficial en el archivo solo para primera columna "Fecha" seria
 como tipo Date y las demas como tipo float.

#### Paso 5: Indagando el contenido de las columas tipo object

Se obvserva que entre los valores de cada columna hay un valor inusual,
 que impide la conversion de las columnas de string a float.

In [ ]:
df_GBA.select_dtypes(include='object')

#### Paso 6: Profundizando en la busqueda del valor inusual

Se opta por intentar realizar la conversion o casteo y generar con la columna "Prendas de vestir y materiales" con la muestra y generar un indicador para comprobar el procedimiento.

In [ ]:
check_cast_number = pd.to_numeric(df_GBA["Prendas de vestir y materiales"], errors='coerce').isnull()
check_cast_number.any()

Al dar verdadero el indicador, procede a identificar el valor inusual y aisladolo por su nivel de fila

In [ ]:
df_GBA[check_cast_number]['Prendas de vestir y materiales']

Una vez, discretado el valor se remplazara por un valor numero como cero (0)
para terminar correctamente el casteo o conversion de los valores de las columnas faltantes  y se comprueba que no quede algun valor inusual del mismo tipo en la muestra de la Region GBA

In [ ]:
df_clean_GBA= df_GBA.replace("///", 0)
df_clean_Pampeana = df_Pampeana.replace("///", 0)
df_clean_Noroeste = df_Noroeste.replace("///", 0)
df_clean_Noreste = df_Noreste.replace("///", 0)
df_clean_Cuyo = df_Cuyo.replace("///", 0)
df_clean_Patagonia = df_Patagonia.replace("///", 0)

df_ckeck_finished = df_clean_GBA.eq("///").any(axis=1).sum()
df_ckeck_finished


### Paso 7: Visualización de los datos 

En esta parte se grafica una muestra con los meses de años 2022

In [ ]:
df_anio_2022 = df_clean_GBA.query("Fecha >= '2022-01-01' and Fecha <= '2022-12-01'")
fig = px.line(df_anio_2022, x="Fecha", y=description_region_principal, labels=description_region_principal)
fig.update_layout(title={"text":'Indices de precios del año 2022', 'y':0.93,'x':0.35, 'xanchor': 'center', 'yanchor': 'top'},
                   xaxis_title='Meses',
                   yaxis_title='Indices',
                   legend_title_text='Rubros:')
fig.show()

Como se puede observar, el rubro mas alto fue el de Prendas de vestir y calzado y el rubro mas bajo fue el de la Comunicacion.